# Build a Verifiable Neural Network with Giza Actions

Giza Actions provides developers with the tools to easily create and expand Verifiable Machine Learning solutions, transforming their Python scripts and ML models into robust, repeatable workflows. Models developed using the Action SDK possess a verifiable property, enabling you to encapsulate your model within a Zero-Knowledge cryptographic layer, thereby ensuring the integrity of the inference.

In this tutorial, we will explore the process of building your first Neural Network using MNIST dataset, [Pytorch](https://pytorch.org/), and Giza Action SDK and demonstrating its verifiability.

# What is MNIST dataset?

The MNIST dataset is an extensive collection of handwritten digits, very popular in the field of image processing. Often, it's used as a reference point for machine learning algorithms. This dataset conveniently comes already partitioned into training and testing sets, a feature we'll delve into later in this tutorial.

The MNIST database comprises a collection of 70,000 images of handwritten digits, ranging from 0 to 9. Each image measures 28 x 28 pixels. For the purpose of this tutorial, we will resize image to 14 x 14 pixels.

<img src="./imgs/mnist_dataset_illustration.png" alt="MNIST Dataset illustration" />


# Login to Giza and create a Workspace

In [1]:
# TODO: remove this cell when it deploys on main branch

import os

os.environ['GIZA_API_HOST'] = 'https://api-dev.gizatech.xyz'
print(os.environ['GIZA_API_HOST'])


https://api-dev.gizatech.xyz


Before we begin, it's important to note that since we will be using Giza tools, you need to log in to the Giza platform. To do this, we recommend following these steps to install the Giza CLI, create a user, and generate API keys:

```bash
$ pipx install giza-cli
$ giza users create
$ giza users login
$ giza users create-api-key
```

For more detailed information about `login`, please refer to the [Giza-CLI documentation](https://cli.gizatech.xyz/welcome/readme).

If you haven't already created a workspace associated with your user, you'll need to do so. Workspaces in our platform are a crucial component designed to enhance user interaction with Giza Actions. These workspaces provide a user-friendly interface (UI) for managing and tracking runs, tasks, and metadata associated with action executions.

Please note that the workspace creation process can take up to 10 minutes as isolated resources are set up for each respective workspace. You can create a workspace using the following command:

```bash
giza workspaces create
```

If you've previously created a workspace with your account, you can retrieve your workspace URL as follows:

```bash
giza workspaces get
```

For more detailed information about `workspaces`, please refer to the [Workspace section of Giza-CLI documentation](https://cli.gizatech.xyz/resources/workspaces).

# Define your model

To begin, we must define the architecture of our model. In this tutorial, we will construct a basic feedforward neural network

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import logging
from scipy.ndimage import zoom
from giza_actions.action import action
from giza_actions.task import task
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
input_size = 196  # 14x14
hidden_size = 10 
num_classes = 10
num_epochs = 10
batch_size = 256
learning_rate = 0.001

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes) 

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

# Prepare your Tasks

In the Giza Action SDK, a Task serves as a functional representation of a specific work segment within a Giza Actions workflow. Tasks offer a means to encapsulate portions of your workflow logic in traceable and reusable units across actions. Essentially, a Giza Actions task can accomplish nearly anything a typical Python function can. The unique aspect of tasks is their ability to access information about upstream dependencies and the status of these dependencies before starting. This capability enables tasks, for instance, to wait for the completion of another task before starting. Moreover, tasks benefit from Giza Actions' automated logging, which captures comprehensive details of each task run, including its duration, tags, and final state.

This is how we will encapsulate the methods we build using the `@task` decorator.

### Prepare datasets and create loaders
We need to download datasets and create loarders for both training and testing purposes.

In [4]:
def resize_images(images):
    return np.array([zoom(image[0], (0.5, 0.5)) for image in images])

@task(name="Prepare Datasets")
def prepare_datasets():
    print("Prepare dataset...")
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False)

    x_train = resize_images(train_dataset)
    x_test = resize_images(test_dataset)

    x_train = torch.tensor(x_train.reshape(-1, 14*14).astype('float32') / 255)
    y_train = torch.tensor([label for _, label in train_dataset], dtype=torch.long)

    x_test = torch.tensor(x_test.reshape(-1, 14*14).astype('float32') / 255)
    y_test = torch.tensor([label for _, label in test_dataset], dtype=torch.long)

    print("✅ Datasets prepared successfully")

    return x_train, y_train, x_test, y_test


In [5]:
@task(name="Create Loaders")
def create_data_loaders(x_train, y_train, x_test, y_test):
    print("Create loaders...")

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

    print("✅ Loaders created!")

    return train_loader, test_loader


### Train the model

We need to define our training method.

In [6]:
@task(name="Train model")
def train_model(train_loader):
    print("Train model...")

    model = NeuralNet(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print("✅ Model trained successfully")
    return model

### Test model

We need to define our testing method.

In [7]:
@task(name="Test model")
def test_model(model, test_loader):
    print("Test model...")
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

# Action the tasks

Now that we've prepared the taks, we need to execute them. This will be done using the `action` decorator.

Think of actions as unique types of functions. They are capable of receiving inputs, executing tasks, and producing outputs. Remarkably, transforming any standard function into a Giza Actions action is as simple as appending the `@action` decorator. This transformation alters the function's characteristics, bestowing several advantages:

- Each time this function is used, its activity is monitored, with every state change communicated to the API for efficient tracking of the action's execution. 
  
- Input parameters undergo automatic type verification and adaptation to ensure they match the required types. 
  
- In cases of failure, the system is equipped to retry. Time constraints can be imposed to avert excessively prolonged workflows.
  
- Actions leverage inherent logging capabilities, which record essential details of each action run, including its duration and conclusive state.

For more detailed information about `actions`, please refer to the Actions section of Action SDK documentation.

In [8]:

@action(name="Execution", log_prints=True)
def execution():
    x_train, y_train, x_test, y_test = prepare_datasets()
    train_loader, test_loader = create_data_loaders(
        x_train, y_train, x_test, y_test)
    model = train_model(train_loader)
    test_model(model, test_loader)


execution()

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NeuralNet`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

When you return to your workspace and refresh it, you will observe your action currently in the execution process.

To view all the actions you have run, navigate to the Action Runs section within your workspace. Each action run represents a single occurrence of executing the action.

<img src="./imgs/action_runs.png" alt="MNIST Dataset illustration" />


Furthermore, by clicking on a particular action, you can closely monitor its progress. You'll have the ability to view logs, individual tasks, and associated metadata.

<img src="./imgs/action.png" alt="MNIST Dataset illustration" />

Giza Action enables you to deploy workflows, turning them from manual activation into API-managed entities that can be triggered remotely. Once a workflow is deployed, it creates a deployment within your Giza Workspace, which remains active, awaiting action runs associated with that deployment. When a run is detected, it is asynchronously executed in a local subprocess.

All your deployments can be found in the 'Deployments' section of your Workspace.

<img src="./imgs/deployments.png" alt="MNIST Dataset illustration" />

# Run and Prove!

Up to this point, we have primarily focused on training and testing our model using PyTorch while monitoring its execution through the Giza platform. However, if you are here, it's likely because you want to harness the capabilities of ZKML (Zero-Knowledge Machine Learning) and have the ability to demonstrate the integrity of your model's inferences.

In this section, we will delve into what it means to prove the integrity of inferences, what setup is required to make this possible, and how to verify the proof.

ZKML leverages validity proofs like SNARKs and STARKs, which enables the verification of the correctness of computational processes. By deploying such proof systems in machine learning applications, we gain the ability to validate the inference of ML models or to confirm that a specific input produced a certain output with a given model.

To generate ZK proofs for your model inferences, you must first convert your model into ZK circuits. This conversion process involves leveraging programming languages that specialize in building ZK circuits, such as [Cairo-lang](https://www.cairo-lang.org/). Subsequently, using the Giza-CLI, you can transpile your model from ONNX to Cairo. This process will be covered in the upcoming sections.

It's worth mentioning that at present, Orion and Action-SDK exclusively supports Cairo as a ZK backend. However, we are actively working on expanding support for other ZK backends (e.g; EZKL, Noir ...).

<img src="./imgs/giza_stack.png" alt="MNIST Dataset illustration" />

## Convert to ONNX

Before invoking the Giza transpiler to convert your model into Cairo, you must first ensure that your model is converted to ONNX. We will explore this process in the following section.

ONNX, short for Open Neural Network Exchange, is an open format for representing and exchanging machine learning models between different frameworks and libraries. It serves as an intermediary format that allows you to move models seamlessly between various platforms and tools, facilitating interoperability and flexibility in the machine learning ecosystem.

In [10]:
import torch.onnx

@task(name="Convert To ONNX")
def convert_to_onnx(model, onnx_file_path):
    dummy_input = torch.randn(1, input_size).to(device)
    torch.onnx.export(model, dummy_input, onnx_file_path,
                      export_params=True, opset_version=10, do_constant_folding=True)

    print(f"Model has been converted to ONNX and saved as {onnx_file_path}")


@action(name="Action: Convert To ONNX", log_prints=True)
def execution():
    x_train, y_train, x_test, y_test = prepare_datasets()
    train_loader, test_loader = create_data_loaders(
        x_train, y_train, x_test, y_test)
    model = train_model(train_loader)
    test_model(model, test_loader)

    # Convert to ONNX
    onnx_file_path = "mnist_model.onnx"
    convert_to_onnx(model, onnx_file_path)


execution()

/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/giza-actions-mYf3m_Lk-py3.11/lib/python3.11/site-packages/prefect/tasks.py:337: UserWarning: A task named 'Convert To ONNX' and defined at '/Users/raphaeldoukhan/Desktop/Orion-Giza/Tools/actions-sdk/giza_actions/task.py:3' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/giza-actions-mYf3m_Lk-py3.11/lib/python3.11/site-packages/prefect/flows.py:338: UserWarning: A flow named 'Action: Convert To ONNX' and defined at '/Users/raphaeldoukhan/Desktop/Orion-Giza/Tools/actions-sdk/giza_actions/action.py:12' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NeuralNet`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

## Transpile your model to Cairo and deploy on Giza

Now we converted to ONNX, use the Giza-CLI to transpile your model to Orion Cairo code. 

```bash
giza transpile mnist_model.onnx --output-path verifiable_mnist
```

After your model has been transpiled, it's ready for deployment on the Giza platform. Our platform's deployment process establishes services capable of accepting prediction requests through a specific endpoint. These services leverage Cairo under the hood to ensure provable inferences.

To create a new service, users can employ the `deploy` command. The following command facilitates the deployment of a machine learning service ready to accept predictions at the `/cairo_run` endpoint.

```bash
giza deployments deploy --model-id 1 --version-id 1 inference.sierra
```

### How to Obtain the Sierra File?

To deploy a Cairo model, you will need to provide its Sierra file. Sierra is an intermediate representation between high level Cairo and compilation targets, such as CASM. To obtain this file, follow these steps:

1. Navigate to your transpiled Cairo model directory:
    ```bash
    $ cd verifiable_mnist/inference
    ```

2. Compile your model using Scarb (make sure to have [Scarb](https://docs.swmansion.com/scarb/download.html) installed):

    ```bash
    $ scarb build
    ```

3. The Sierra file can be found in the generated directory at `target/dev/inference.sierra`.

Please keep in mind that if your model contains ONNX operators not currently supported by the Transpiler, the transpilation process will fail. We are actively working towards achieving 100% ONNX compatibility in both Orion and the Transpiler.

## Run your program

Now that your Cairo model is deployed on the Giza platform, you have the capability to execute it. If you initiate a prediction using Giza Action without the `verifiable` mode, it runs the onnx version of the model.


When you initiate a prediction using Giza Action in `verifiable` mode, it executes the Sierra program using CairoVM, generating trace and memory files for the proving. It also returns the output value and initiates a proving job to generate a Stark proof of the inference.



First, let's make a prediction with the ONNX model (`veriable=False`) 

In [12]:
from giza_actions.model import GizaModel

model_id = 49  # Update with your model ID
version_id = 2  # Update with your version ID


@task(name='Preprocess Image')
def preprocess_image(image_path):
    from PIL import Image
    import numpy as np

    # Load image, convert to grayscale, resize and normalize
    image = Image.open(image_path).convert('L')
    # Resize to match the input size of the model
    image = image.resize((14, 14))
    image = np.array(image).astype('float32') / 255
    image = image.reshape(1, 196)  # Reshape to (1, 196) for model input
    return image


@task(name='Prediction with ONNX')
def prediction(image, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    result = model.predict(
        input_feed={"image": image}, verifiable=False, output_dtype="tensor_fixed_point"
    )

    return result


@action(name='Execution: Prediction with ONNX', log_prints=True)
def execution():
    image = preprocess_image("./zero.jpg")
    result = prediction(image, model_id, 1)
    print("Result: ", result)

    return result


execution()

/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/giza-actions-mYf3m_Lk-py3.11/lib/python3.11/site-packages/prefect/tasks.py:337: UserWarning: A task named 'Preprocess Image' and defined at '/Users/raphaeldoukhan/Desktop/Orion-Giza/Tools/actions-sdk/giza_actions/task.py:7' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/giza-actions-mYf3m_Lk-py3.11/lib/python3.11/site-packages/prefect/tasks.py:337: UserWarning: A task named 'Prediction with ONNX' and defined at '/Users/raphaeldoukhan/Desktop/Orion-Giza/Tools/actions-sdk/giza_actions/task.py:21' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/giza-actions-mYf3m_Lk-py3.11/lib/python3.11/site-packages/prefect

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `ndarray`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

Now, let's make a prediction with the Cairo model (`veriable=True`).

DISCLAIMER: This section is still a work in progress and may not function correctly with certain Cairo programs. We are diligently working to resolve any issues and ensure it operates smoothly as soon as possible.

In [13]:
@task(name='Prediction with Cairo')
def prediction(image, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    result = model.predict(
        input_feed={"image": image}, verifiable=True, output_dtype="tensor_fixed_point"
    )

    return result


@action(name='Execution: Prediction with Cairo', log_prints=True)
def execution():
    image = preprocess_image("./zero.jpg")
    result = prediction(image, model_id, 1)
    print("Result: ", result)

    return result


execution()

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `ndarray`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

TODO: prover and verifier section